In [1]:
!pip install librosa soundfile numpy tqdm

     -------------------------------------- 260.7/260.7 kB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 6.4 MB/s eta 0:00:00
     ---------------------------------------- 2.8/2.8 MB 3.8 MB/s eta 0:00:00
     ---------------------------------------- 64.6/64.6 kB 3.6 MB/s eta 0:00:00
     -------------------------------------- 166.7/166.7 kB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 30.3/30.3 MB 6.3 MB/s eta 0:00:00


In [2]:
import os
import librosa
import soundfile as sf
import numpy as np
from tqdm import tqdm

### Preprocessing Audio

1. Resampling ke sample rate yang seragam (16.000 Hz)
2. Normalisasi audio
3. Potong atau padding audio ke durasi tetap (5 detik)

In [3]:
# Parameter global
TARGET_SR = 16000       # Target sample rate
DURATION = 5            # Durasi target (detik)
SAMPLES = TARGET_SR * DURATION  # Total sample

# Fungsi normalisasi + resample + padding/trim
def preprocess_audio(file_path):
    y, sr = librosa.load(file_path, sr=TARGET_SR)
    
    # Padding atau trimming
    if len(y) < SAMPLES:
        y = np.pad(y, (0, SAMPLES - len(y)))
    else:
        y = y[:SAMPLES]
    
    # Normalisasi
    y = y / np.max(np.abs(y))
    
    return y


In [4]:
input_root = "dataset"
output_root = "processed_dataset"

os.makedirs(output_root, exist_ok=True)

for label in os.listdir(input_root):
    label_path = os.path.join(input_root, label)
    
    if os.path.isdir(label_path):
        os.makedirs(os.path.join(output_root, label), exist_ok=True)
        
        for file in tqdm(os.listdir(label_path), desc=f"Processing {label}"):
            if file.endswith(".wav"):
                input_path = os.path.join(label_path, file)
                output_path = os.path.join(output_root, label, file)
                
                try:
                    y = preprocess_audio(input_path)
                    sf.write(output_path, y, TARGET_SR)
                except Exception as e:
                    print(f"❌ Gagal proses {file}: {e}")


Processing silence: 100%|██████████| 108/108 [00:01<00:00, 60.49it/s]
